In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import pickle
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
!ls

 0_read_data.ipynb	    2_app_pruebas.ipynb   __pycache__
'1_join_data_&_EDA.ipynb'   helper_functions.py


In [3]:
# seteo carpetas
BASE = Path().absolute().parent
RAW = Path(BASE/'raw')
DATA = Path(BASE/'data')

In [4]:
def load_datasets():
    df = pickle.load( open( DATA/"df_fide.p", "rb" ) )
    df['antig'] = [round((datetime.now() - x).days/365) for x in df['fecha_ingreso']]
    df['edad'] = [np.nan if pd.isnull(x) else round((datetime.now() - x).days/365) for x in df['fecha_nac']]
    return df

In [5]:
df_socios = load_datasets()

In [6]:
print(df_socios.columns)
# df.head()

Index(['sexo', 'tipo_socio', 'forma_pago', 'fecha_ingreso', 'refunda',
       'fecha_nac', 'abo_bid_2020', 'abo_bid_2021', 'abo_pol_2020',
       'abo_pol_2021', 'eve_2018', 'eve_2019', 'eve_2020', 'eve', 'partidos',
       'antig', 'edad', 'grupo_etario'],
      dtype='object')


In [7]:
# seteo valores random
patrimonial =  1500
refundador = 2000
debito = 1000
ant_simple = 250
ant_pleno = 350
ant_interior = 350
ant_exterior = 500
ant_vitalic = 350
abo_bidegain = 1000
abo_polideportivo = 500
vita_abonando = 25
cuota_dia_aspo = 1000
eve_bid_amba = 50
eve_pol_inte = 75
eve_bid_inte = 50
eve_pol_amba = 75

In [13]:
def calc_puntos(dfinput, patrimonial, refundador, debito, ant_simple, ant_pleno, ant_interior, \
                ant_exterior, ant_vitalic, abo_bidegain, abo_polideportivo, eve_bid_amba, eve_bid_inte, \
                cancha_mujer = 1):
    df = dfinput.copy()
    df['puntos'] = 0

    df.loc[df['tipo_socio']=='PATRIMONIAL', 'puntos'] += patrimonial
    df.loc[df['refunda']==1, 'puntos'] += refundador
    df.loc[df['forma_pago']=='DEBITO', 'puntos'] += debito

    df.loc[df['tipo_socio']=='SIMPLE', 'puntos'] += df.loc[df['tipo_socio']=='SIMPLE', 'antig'] * ant_simple
    df.loc[df['tipo_socio']=='PLENO', 'puntos'] += df.loc[df['tipo_socio']=='PLENO', 'antig'] * ant_pleno
    df.loc[df['tipo_socio']=='INTERIOR', 'puntos'] += df.loc[df['tipo_socio']=='INTERIOR', 'antig'] * ant_interior
    df.loc[df['tipo_socio']=='EXTERIOR', 'puntos'] += df.loc[df['tipo_socio']=='EXTERIOR', 'antig'] * ant_exterior
    df.loc[df['tipo_socio']=='VITALICIO', 'puntos'] += df.loc[df['tipo_socio']=='VITALICIO', 'antig'] * ant_vitalic

    df['puntos'] += df['abo_bid_2020'] * abo_bidegain
    df['puntos'] += df['abo_bid_2021'] * abo_bidegain
    df['puntos'] += df['abo_pol_2020'] * abo_polideportivo
    df['puntos'] += df['abo_pol_2021'] * abo_polideportivo

    intext = (df['tipo_socio']=='INTERIOR') | (df['tipo_socio']=='EXTERIOR')
    df.loc[((~intext) & (df['sexo']=='M')), 'puntos'] += df.loc[~intext, 'eve_2018'] * eve_bid_amba
    df.loc[((~intext) & (df['sexo']=='M')), 'puntos'] += df.loc[~intext, 'eve_2019'] * eve_bid_amba
    df.loc[((~intext) & (df['sexo']=='M')), 'puntos'] += df.loc[~intext, 'eve_2020'] * eve_bid_amba

    df.loc[((~intext) & (df['sexo']=='F')), 'puntos'] += (df.loc[~intext, 'eve_2018'] * eve_bid_amba * cancha_mujer)
    df.loc[((~intext) & (df['sexo']=='F')), 'puntos'] += (df.loc[~intext, 'eve_2018'] * eve_bid_amba * cancha_mujer)
    df.loc[((~intext) & (df['sexo']=='F')), 'puntos'] += (df.loc[~intext, 'eve_2018'] * eve_bid_amba * cancha_mujer)
    
    df.loc[((intext) & (df['sexo']=='M')), 'puntos'] += df.loc[intext, 'eve_2018'] * eve_bid_inte
    df.loc[((intext) & (df['sexo']=='M')), 'puntos'] += df.loc[intext, 'eve_2019'] * eve_bid_inte
    df.loc[((intext) & (df['sexo']=='M')), 'puntos'] += df.loc[intext, 'eve_2020'] * eve_bid_inte

    df.loc[((intext) & (df['sexo']=='F')), 'puntos'] += (df.loc[intext, 'eve_2018'] * eve_bid_inte * cancha_mujer)
    df.loc[((intext) & (df['sexo']=='F')), 'puntos'] += (df.loc[intext, 'eve_2019'] * eve_bid_inte * cancha_mujer)
    df.loc[((intext) & (df['sexo']=='F')), 'puntos'] += (df.loc[intext, 'eve_2020'] * eve_bid_inte * cancha_mujer)
    
    return df

In [14]:
df_socios['sexo'].value_counts()

M    45637
F    11012
Name: sexo, dtype: int64

In [15]:
df1 = calc_puntos(df_socios, patrimonial, refundador, debito, ant_simple, ant_pleno, ant_interior, \
        ant_exterior, ant_vitalic, abo_bidegain, abo_polideportivo, eve_bid_amba, eve_bid_inte)
df1.groupby(['sexo'])['puntos'].mean()

sexo
F    3472.625318
M    4450.847996
Name: puntos, dtype: float64

In [17]:
df2 = calc_puntos(df_socios, patrimonial, refundador, debito, ant_simple, ant_pleno, ant_interior, \
        ant_exterior, ant_vitalic, abo_bidegain, abo_polideportivo, eve_bid_amba, eve_bid_inte, \
        cancha_mujer = 2)
df2.groupby(['sexo'])['puntos'].mean()

sexo
F    3588.839448
M    4450.847996
Name: puntos, dtype: float64

In [ ]:
df_socios.groupby(['sexo'])['puntos'].mean()

In [ ]:
# round(df.puntos.describe(),1)

In [ ]:
# fig = px.histogram(df, x="puntos", nbins=20, labels={"puntos":'Puntos', 'count':'# Socios'}).update_layout(yaxis_title="# Socios")
# fig.show()

In [ ]:
df['edad'].describe()

In [ ]:
round(df[df['tipo_socio']=='SIMPLE'].shape[0]/df.shape[0]*100)

In [ ]:
df['eve'] = df['eve_2018'] + df['eve_2019'] + df['eve_2020']
df['eve'].describe()

In [ ]:
df.columns

In [ ]:
df['antig'].head()

In [ ]:
df['partidos'].unique()

In [ ]:
# tipo de socios
# tipo_socio = list(df['tipo_socio'].unique())
tipo_socio = ['TODOS'] + list(df['tipo_socio'].unique())
tipo_socio